In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, RFECV, mutual_info_classif, mutual_info_regression, SelectPercentile
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sn


import warnings; warnings.simplefilter('ignore')
%matplotlib inline

# Data Preprocessing
Introduce the Data¶

Task: Given EEG series, predict wheiter they have Epileptic-Seizure or not

In [ ]:
df = pd.read_csv('data.csv')

df.head()

Binary Classification

In [ ]:
cols = df.columns
outcome = df.y
outcome[outcome>1]=0
df.head()

Outcome Plot

In [ ]:
ax = sn.countplot(outcome,label="Count")
non_seizure, seizure = outcome.value_counts()
print('O número de testes para a classe sem crises é:', non_seizure)
print('O número de testes para a classe com crises é:', seizure)

Pandas Indexation

In [ ]:
X = df.iloc[:,1:179].values
X.shape

In [ ]:
y = df.iloc[:,179].values
y

# Feature election and model building

   ## Splitting the dataset into the Training set and Test set

In [ ]:
#from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
#Classificadores
modelos = []
modelos.append(('Logistic Regression', LogisticRegression(solver='liblinear', multi_class='ovr')))
modelos.append(('Linear Discriminant Analysis', LinearDiscriminantAnalysis()))
modelos.append(('K-Nearest Neighbors', KNeighborsClassifier()))
modelos.append(('Classification and Regression Trees', DecisionTreeClassifier()))
modelos.append(('Gaussian Naive Bayes', GaussianNB()))
modelos.append(('Linear Support Vector Machines', svm.LinearSVC()))
modelos.append(('Extra Trees Classifier',ExtraTreesClassifier()))

In [ ]:
resultados = []
nomes = []
for nome, modelo in modelos:
	resultado_validacao_cruzada = cross_val_score(modelo, X_train, y_train, cv=kfold, scoring='accuracy')
	resultados.append(resultado_validacao_cruzada)
	nomes.append(nome)
	print('%s: %f (%f)' % (nome, resultado_validacao_cruzada.mean(), resultado_validacao_cruzada.std()))

In [ ]:
plt.boxplot(resultados)
plt.title('Comparação de Algoritmos')
plt.show()

In [ ]:
predicao = []
for nome, modelo in modelos:
    modelo.fit(X_train,y_train)
    predicao.append(modelo.predict(X_test))

In [ ]:
for i in range(len(predicao)):
    print(nomes[i])
    print(accuracy_score(y_test,predicao[i]))
    print(confusion_matrix(y_test,predicao[i]))
    print(classification_report(y_test,predicao[i]))